In [154]:
import numpy as np
import matplotlib.pyplot as plt
import subprocess
from configobj import ConfigObj
from itertools import chain
import os, glob
import shutil
import pandas as pd
from astropy.io import fits
from astropy.table import Table

In [190]:
#define paramters to be modified in default config file
#for any section that requires paramters to be edited, include it in 'sed_modules_params'
CIGALE_PARAMS = {'data_file': 'ztf_fbots_cigale.txt',
                 'sed_modules': ['sfhdelayed', 'bc03', 'nebular', 'dustatt_calzleit', 'dale2014', 'redshifting'],
                 'analysis_method': "pdf_analysis",
                 'sed_modules_params': {
                     'sfhdelayed': {
                        'tau_main': ['250', '500', '1000', '2000', '4000', '6000', '8000'],
                        'age_main': ['250', '500', '1000', '2000', '4000', '8000', '12000'],
                     },
                     'bc03': {
                         'imf': 1,
                         'metallicity': ['0.0001', '0.0004', '0.004', '0.008', '0.02', '0.05'],
                     },
                     'dustatt_calzleit': {
                         'E_BVs_young': ['0.0', '0.2', '0.3', '0.5', '0.8', '1.0', '1.5', '2.0', '3.0'],
                         'E_BVs_old_factor': ['0.3', '0.5', '1.0'],
                         'uv_bump_amplitude': ['0.0', '1.0', '2.0', '3.0'],
                         'powerlaw_slope': ['-0.13', '-0.2', '-0.5']
                     },
                     'dale2014': {
                         'alpha': ['1.0', '1.5', '2.0', '2.5'],
                     }
                 },
                 'analysis_params': {
                     'save_best_sed': True,
                 }}

In [191]:
def prep_cigale(params):

    subprocess.run(["pcigale", "init"])
    configfile = 'pcigale.ini'

    #read in configfile
    config = ConfigObj(configfile, encoding = 'utf-8')
    config.filename = configfile

    #edit params
    config['data_file'] = params['data_file']
    config['sed_modules'] = params['sed_modules']
    config['analysis_method'] = params['analysis_method']
    config.write()
    
    #generate config file to run cigale
    try:
        subprocess.run(["pcigale", "genconf"])
    except:
        raise ValueError("Could not create config file - check defaults")
    
    #read configfile again as it has been modified by genconf
    config = ConfigObj(configfile, encoding = 'utf-8')
    config.filename = configfile
    
    #genconf will overwrite any edits, so modify paramters at the end
    for module_ in params['sed_modules']:
        if module_ in params['sed_modules_params'].keys():
            mod = params['sed_modules_params'][module_]
            for key, value in mod.items():
                config['sed_modules_params'][module_][key] = value

    if 'analysis_params' in params.keys():
        for key, value in params['analysis_params'].items():
            config['analysis_params'][key] = value
    config.write()

    #verify if config file is ok
    try:
        subprocess.run(["pcigale", "check"])
    except:
        raise ValueError("Config file in incorrect")
    
    return config

In [192]:
def run_cigale(configfile, params, outdir, plot=False):
    if not os.path.exists(outdir):
        os.makedirs(outdir)

    #copy everything to outdir
    shutil.copy(configfile, os.path.join(outdir, configfile))
    shutil.copy(configfile + '.spec', os.path.join(outdir, configfile + '.spec'))
    shutil.copy(params['data_file'], os.path.join(outdir, params['data_file']))

    cwd = os.getcwd()
    os.chdir(outdir)

    #run cigale
    subprocess.run(["pcigale", "run"])
    if plot:
        subprocess.run(["pcigale-plots", "sed"])
    os.chdir(cwd)

In [193]:
def read_ptf_sample(filename):
    try:
        ptf_sne = pd.read_csv(filename, header = None, names=['SN', 'telescope', 'instrument', 'filter', 'mag', 'magerr', 'ref'])
        ptf_sne['SN'] = ptf_sne['SN'].str.strip()
        ptf_sne['telescope'] = ptf_sne['telescope'].str.strip()
        ptf_sne['instrument'] = ptf_sne['instrument'].str.strip()
        ptf_sne['filter'] = ptf_sne['filter'].str.strip()
        ptf_sne['ref'] = ptf_sne['ref'].str.strip()
        
        return ptf_sne
    except:
        raise ValueError('Reformat input file with correct whitespaces and delimiters')

In [194]:
config = prep_cigale(CIGALE_PARAMS)

╭──────────────────────────────────────────────────────────────────────────────╮
│                      Code Investigating GALaxy Emission                      │
│                Boquien et al. (2019) (]8;id=507815;https://cigale.lam.fr\https://cigale.lam.fr]8;;\)                 │
│         CIGALE version: 2025.0 — Python version: 3.12.0 — Platform:          │
│                              macosx-11.1-arm64                               │
╰───────────────────────────���──────────────────────────────────────────────────╯


                                  SED modules                                   
╭──────────────────────┬───────────────────────────────────────────────────────╮
│ SFH                  │ None. Options are: sfh2exp, sfh_buat08, sfhdelayed,   │
│                      │ sfhdelayedbq, sfhfromfile, sfhperiodic.               │
│ SSP                  │ None. Options are: bc03, bpassv2, cb19, m2005.        │
│ nebular              │ None. Options are: nebular.         

In [195]:
run_cigale('pcigale.ini', CIGALE_PARAMS, 'sn2018bcc', plot=True)

╭──────────────────────────────────────────────────────────────────────────────╮
│                      Code Investigating GALaxy Emission                      │
│                Boquien et al. (2019) (]8;id=242611;https://cigale.lam.fr\https://cigale.lam.fr]8;;\)                 │
│         CIGALE version: 2025.0 — Python version: 3.12.0 — Platform:          │
│                              macosx-11.1-arm64                               │
╰───────────────────────────���──────────────────────────────────────────────────╯


                              General information                               
            ╭───────────────────┬──────────────────────────────────╮            
            │ Data file         │ ztf_fbots_cigale.txt             │            
            │ Parameters file   │ None                             │            
            │ Number of objects │ 1                                │            
            │ Redshift          │ 0.06 to 0.06               

In [163]:
ptf_sne = read_ptf_sample('ptf_ccsne.csv')
ptf_sne = ptf_sne[ptf_sne['ref'] == 'This paper']

In [91]:
sn = 'PTF09awk'
ptf09awk = ptf_sne[ptf_sne['SN'] == sn]
awkmag, awkerr = ptf09awk['mag'].to_numpy(), ptf09awk['magerr'].to_numpy()
awkflux = 10**((awkmag - 8.90)/-2.5)*1000
awkfluxerr = awkerr * awkflux * np.log(10)/2.5

In [ ]:
awkmag = np.array([20.00, 19.90, 19.55, 18.53, 18.24, 17.96, 17.93, 18.75, 19.52])
awkerr = np.array([0.17, 0.06, 0.11, 0.03, 0.06, 0.09, 0.09, 0.14, 0.12])
awkflux = 10**((awkmag - 8.90)/-2.5)*1000
awkfluxerr = awkerr * awkflux * np.log(10)/2.5

In [94]:
ptf09awk

,SN,telescope,instrument,filter,mag,magerr,ref
16,PTF09awk,GALEX,,FUV,20.24,0.26,This paper
17,PTF09awk,GALEX,,NUV,20.03,0.10,This paper
18,PTF09awk,SDSS,,u,19.20,0.06,This paper
19,PTF09awk,SDSS,,g,18.16,0.05,This paper
20,PTF09awk,SDSS,,r,17.80,0.05,This paper
21,PTF09awk,SDSS,,i,17.51,0.05,This paper
22,PTF09awk,SDSS,,z,17.43,0.06,This paper
23,PTF09awk,PANSTARRS,,g,18.15,0.05,This paper
24,PTF09awk,PANSTARRS,,r,17.86,0.05,This paper
25,PTF09awk,PANSTARRS,,i,17.52,0.05,This paper


In [165]:
line = [f'{i} {j}' for i, j in zip(awkflux, awkfluxerr)]
' '.join(line)

'0.03630780547701017 0.005684923192247002 0.039810717055349776 0.002200021527193253 0.05495408738576248 0.005567604346196149 0.1406047524129913 0.0038850528829232683 0.18365383433483493 0.010149085949054136 0.2376840286624876 0.019702357244676484 0.2443430552693974 0.020254344359037647 0.1148153621496883 0.014804839914624765 0.05649369748123034 0.006243914192083021'

In [221]:
hdul = fits.open('sn2018bcc/out/results.fits')
results = Table(hdul[1].data)

In [222]:
m_star, m_star_err = results['bayes.stellar.m_star'].value[0], results['bayes.stellar.m_star_err'].value[0]
log_m_star, log_m_star_err = np.log10(m_star), m_star_err/m_star
print(log_m_star, log_m_star_err)

9.23787901409822 0.3397904423387466


In [224]:
sfh, sfh_err = results['bayes.sfh.sfr'].value[0], results['bayes.sfh.sfr_err'].value[0]
log_sfh, log_sfh_err = np.log10(sfh), sfh_err/sfh
print(log_sfh, log_sfh_err)

-0.528113197485382 0.6030976064028418


In [232]:
age, age_err = results['bayes.sfh.age'].value[0]*1e6, results['bayes.sfh.age_err'].value[0]*1e6
log_age, log_age_err = np.log10(age), age_err/age
print(log_age, log_age_err)

9.834795331399464 0.5293698069209929
